In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('ProblemCData.csv')
material_codes = ['FFTCB', 'PATCB', 'EMTCB', 'MGTCB', 'JFTCB', 'NGTCB','CLTCB','WWTCB', 'DFTCB', 'RFTCB',
'NUEGB','RETCB','HYTCB','GETCB','SOTCB','WYTCB', 'BMTCB'] # list of material codes from excel[]

# Data should be a list with len(material_codes (~20)) that is a list of values
data = df.loc[df['MSN'].isin(material_codes)]
print(data.head())

        MSN StateCode  Year        Data
4720  BMTCB        AZ  1960  4012.94849
4721  BMTCB        AZ  1961  3836.96290
4722  BMTCB        AZ  1962  3672.12878
4723  BMTCB        AZ  1963  4027.53031
4724  BMTCB        AZ  1964  4089.10173


In [22]:
def get_materials_numbers(data):
    dfPA = data[data.loc[:, 'MSN']== 'PATCB']
    dfEM = data[data.loc[:, 'MSN']== 'EMTCB']
    dfJF = data[data.loc[:, 'MSN']== 'JFTCB']
    dfMG = data[data.loc[:, 'MSN']== 'MGTCB']
    dfRF = data[data.loc[:, 'MSN']== 'RFTCB']
    dfDF = data[data.loc[:, 'MSN']== 'DFTCB']
    petro = dfPA.loc[:, 'Data'].values + dfEM.loc[:, 'Data'].values + dfJF.loc[:, 'Data'].values + dfMG.loc[:, 'Data'].values + dfRF.loc[:, 'Data'].values + dfDF.loc[:, 'Data'].values

    dfNG = data[data.loc[:, 'MSN']== 'NGTCB']
    natural_gas = dfNG.loc[:, 'Data'].values
    
    dfCoal = data[data.loc[:, 'MSN']== 'CLTCB']
    dfWood = data[data.loc[:, 'MSN']== 'WWTCB'] 
    coal_wood = dfCoal.loc[:, 'Data'].values + dfWood.loc[:, 'Data'].values 
    
    dfNuc = data[data.loc[:, 'MSN']== 'NUEGB'] 
    nuclear = dfNuc.loc[:, 'Data'].values
    
    dfWind = data[data.loc[:, 'MSN']== 'WYTCB'] 
    wind = dfWind.loc[:, 'Data'].values
    
    dfSol = data[data.loc[:, 'MSN']== 'SOTCB'] 
    solar = dfSol.loc[:, 'Data'].values
    
    dfHydro = data[data.loc[:, 'MSN']== 'HYTCB'] 
    hydro = dfHydro.loc[:, 'Data'].values
    
    dfGeo = data[data.loc[:, 'MSN']== 'GETCB'] 
    geo = dfGeo.loc[:, 'Data'].values
    
    dfBio = data[data.loc[:, 'MSN']== 'BMTCB'] 
    bio = dfBio.loc[:, 'Data'].values
    
    total = petro + natural_gas + coal_wood + nuclear + wind + solar + hydro + geo + bio
    
    data_values = [petro, natural_gas, coal_wood, nuclear, wind, solar, hydro, geo, bio, total]
    #data = []
    #data.extend(data_values)
    return data_values
    
#get_materials_numbers(data)

In [23]:
def state_df(df, state):
    state_df = df[df['StateCode']== state]
    return state_df

def year_df(df, year):
    year_df = df[df['Year']== year]
    return year_df


In [24]:
# Get the values for 2009 for all the codes for all the states
df2009 = year_df(df, 2009)
df2009.name = '2009 Data'


dfCA = state_df(df, 'CA')
dfCA2009 = year_df(dfCA, 2009)
dfCA2009.name = 'California 2009 Data'

dfAZ = state_df(df, 'AZ')
dfAZ2009 = year_df(dfAZ, 2009)
dfAZ2009.name = 'Arizona 2009 Data'

dfNM = state_df(df, 'NM')
dfNM2009 = year_df(dfNM, 2009)
dfNM2009.name = 'New Mexico 2009 Data'

dfTX = state_df(df, 'TX')
dfTX2009 = year_df(dfTX, 2009)
dfTX2009.name = 'Texas 2009 Data'


In [25]:
import itertools
def make_profile_df_unrefined(df):
    data_abs = get_materials_numbers(df)
    total = sum(data_abs)
    
    data_rel = []
    for i in range(len(data_abs)):
        data_rel.append((data_abs[i] / total) *2)
    materials = ['Petroleum and Oil', 'Natural Gas', 'Coal and Wood', 'Nuclear', 'Wind', 'Solar', 'Hydroelectric', 'Geothermal', 'Biomass', 'total']
    inputs = {'Usage in BTUs': data_abs, 'Percenage of Total Energy Consumed': data_rel}
    data = pd.DataFrame(inputs, index=materials)
    return data

fd = make_profile_df_unrefined(dfCA2009)
fd

,Percenage of Total Energy Consumed,Usage in BTUs
Petroleum and Oil,[0.654131219086],[6863216.01832]
Natural Gas,[0.227921461678],[2391376.808]
Coal and Wood,[0.0183530570227],[192562.27385]
Nuclear,[0.0316665973809],[332249.3898]
Wind,[0.00543232799916],[56996.57722]
Solar,[0.00299243989738],[31397.00543]
Hydroelectric,[0.025942089993],[272187.2346]
Geothermal,[0.0121482841158],[127461.1205]
Biomass,[0.0214125228272],[224662.5224]
total,[1.0],[10492108.9501]


In [38]:
def turn_profile_df_lists_to_numbers(df):
    s = df.loc[:, 'Percenage of Total Energy Consumed']
    percentage_list = []
    for x in s:
        percentage_list.extend(x*100)

    t = fd.loc[:, 'Usage in BTUs']
    usage_list = []
    for y in t:
        usage_list.extend(y)
    return usage_list, percentage_list

usage_list, percentage_list = turn_profile_df_lists_to_numbers(fd)

In [48]:
def make_profile_df(usage_list, percentage_list):
    pd.options.display.float_format = '{:,.2f}'.format
     
    materials = ['Petroleum and Oil', 'Natural Gas', 'Coal and Wood', 'Nuclear', 'Wind', 'Solar', 'Hydroelectric', 'Geothermal', 'Biomass', 'Total']
    inputs = {'Usage in BTUs': usage_list, 'Percentage of Total Energy Consumed': percentage_list}
    data = pd.DataFrame(inputs, index=materials).sort_values(by=['Percentage of Total Energy Consumed'], ascending=False)
    data.columns.name = 'California Energy Profile for 2009'

    return data

make_profile_df(usage_list, percentage_list)



California Energy Profile for 2009,Percentage of Total Energy Consumed,Usage in BTUs
Total,100.00,"10,492,108.95"
Petroleum and Oil,65.41,"6,863,216.02"
Natural Gas,22.79,"2,391,376.81"
Nuclear,3.17,"332,249.39"
Hydroelectric,2.59,"272,187.23"
Biomass,2.14,"224,662.52"
Coal and Wood,1.84,"192,562.27"
Geothermal,1.21,"127,461.12"
Wind,0.54,"56,996.58"
Solar,0.30,"31,397.01"


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas.tools.plotting import table
def save_df_to_png(df):
    ax = plt.subplot(111, frame_on=False) # no visible frame
    ax.xaxis.set_visible(False)  # hide the x axis
    ax.yaxis.set_visible(False)  # hide the y axis

    table(ax, df)  # where df is your data frame

    plt.savefig('mytable.png')

In [14]:
# For each state, the energy profile for the year 2009
make_profile_df(dfCA2009)
make_profile_df(dfNM2009)
make_profile_df(dfAZ2009)
make_profile_df(dfTX2009)

ValueError: Shape of passed values is (2, 40), indices imply (2, 10)

In [11]:
# Plot relative sources of energy used in a given state for a given year
def plot_sources_pie(df):
    profile_df = make_profile_df(df)
    percentages = profile_df.loc[:, 'Percentage of Total Energy Consumed']
    #print(percentages)
    
    materials = ['Petroleum and Oil', 'Natural Gas', 'Coal and Wood', 'Nuclear', 'Wind', 'Solar', 'Other']
    print('Where Other is Hydroelectric, Geothermal, & Biomass')
    colors = ['red', 'red', 'red', 'red', 'green', 'green', 'green']
    scaling = 5
    
    sizes = [item*scaling for item in percentages[:-1]]
    print(len(sizes))
    # TODO switch out the sizes and use the one representing the actual data
    
    #sizes = proportion * scaling
    #sizes = [215, 130, 245, 210, 215, 130, 245, 210]
    explode = (0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1)  # explode 1st slice

    plt.pie(sizes, explode=explode, labels=materials, colors=colors,
            autopct='%5.1f%%', shadow=True, startangle=140)
    plt.axis('equal')
    plt.show()
    
plot_sources_pie(dfCA2009)

NameError: name 'usage_list' is not defined

In [511]:
def plot_state_sources_over_time(df):
    ''' Make a line chart showing a states relative use of renewable and non renewable energy'''
    year = range(1969, 2009) 
    print('Length of year list is {}'.format(len(year)))
    
    ff_total_use = df.loc['Fossiel Fuel']
    rn_total_use = df.loc['Renewable']
    total_usage = ff_total_use + rn_total_use
    
    ff_rel_use = [i / total_usage for i in df.loc['Fossil Fuel']]
    rn_rel_use = [i / total_usage for i in df.loc['Renewable']]
    
    plt.plot(year, ff_rel_use, color='red')
    plt.plot(year, rn_rel_use, color='green')
    plt.xlabel('Time')
    plt.ylabel('Percentage of Total Energy')
    plt.title('{State} use of energy over time')
    plt.show()

plot_state_sources_over_time(df)

Length of year list is 40


KeyError: 'the label [Fossiel Fuel] is not in the [index]'

In [ ]:
s = fd.loc[:, 'Percenage of Total Energy Consumed']
percentage_list = []
for x in s:
    percentage_list.extend(x*100)
    
t = fd.loc[:, 'Usage in BTUs']
usage_list = []
for y in t:
    usage_list.extend(y

In [422]:
# make a dataframe for each state that holds the energy categories
labels = ['Petroleum', 'Natural Gas', 'Coal and Wood', 'Wind', 'Solar', 'Hydroelectric', 'Geothermal', 'Biomass']
data = [1, 4, 6, 3, 67, 7, 7, 7]
df_mat_time = pd.DataFrame(data, labels)

# How can we get all slices of data for a state, for a given year, grab each of the things that matchthe codes and put them ino a vector

In [ ]:
dfDF = data[data.loc[:, 'MSN']== 'DFTCB']
#print(dfDF)
natural_gas = dfPA.loc[:, 'Data'].values

In [ ]:
petro_inputs = data[data.loc[:, 'MSN']== 'PATCB'], data[data.loc[:, 'MSN']== 'NGTCB'], data[data.loc[:, 'MSN']== 'NGTCB'] + data[data.loc[:, 'MSN']== 'NGTCB']
#petro = petro_inputs['Data']
#print(petro['Data'])


'''
natural_gas = data[data.loc[:, 'MSN']== 'NGTCB']] 
coal_wood = data.loc[:, 'MSN']== 'CLTCB'] + data.loc[:, 'MSN']== 'WWTCB']
nuclear = data.loc[:, 'MSN']== 'NUECB']
wind = data.loc[:, 'MSN']== 'WYTCB']
solar = data.loc[:, 'MSN']== 'SOTCB']
hydroelectric = data.loc[:, 'MSN']== 'HYTCB']
geothermal = data.loc[:, 'MSN']== 'GETCB']
biomass = data.loc[:, 'MSN']== 'BMTCB']
'''

In [ ]:
# Make df a dataframe that represents the summary data from excel in a way we can query nicely
#df_CA = data[data['StateCode']== 'CA']
#data[data['StateCode']== 'AZ']
#df9[df9['StateCode']== 'CA']

In [322]:
#df9['MSN'].astype(str).str[0:3].head()

In [335]:
#groupedCA = df9_CA.groupby(df9_CA['MSN'].astype(str).str[0:3]).sum().drop(columns ='Year')
#groupedAZ = df9_CA.groupby(df9_AZ['MSN'].astype(str).str[0:3]).sum().drop(columns ='Year')
#groupedNM = df9_CA.groupby(df9_NM['MSN'].astype(str).str[0:3]).sum().drop(columns ='Year')
#groupedTX = df9_CA.groupby(df9_TX['MSN'].astype(str).str[0:3]).sum().drop(columns ='Year')

In [337]:
'''
data_condensed = [data['PATCB'] + data['EMTCB'] + data['MGTCB'] + data['JFTCB'] + data['DFTCB'] + data['RFTCB'], 
                  data['NGTCB'],
                  data['CLTCB']  + data['WWTCB'],
                  data['NUEGB'],
                  data['WYTCB'],
                  data['SOTCB'],
                  data['HYTCB'],
                  data['GETCB'],
                  data['BMTCB']]
'''

"\ndata_condensed = [data['PATCB'] + data['EMTCB'] + data['MGTCB'] + data['JFTCB'] + data['DFTCB'] + data['RFTCB'], \n                  data['NGTCB'],\n                  data['CLTCB']  + data['WWTCB'],\n                  data['NUEGB'],\n                  data['WYTCB'],\n                  data['SOTCB'],\n                  data['HYTCB'],\n                  data['GETCB'],\n                  data['BMTCB']]\n"

In [408]:
#df9_CA['MSN'].astype(str).str[0:3].head()
#df9_CA = df9[df9['StateCode']== 'CA']
#df9_AZ = df9[df9['StateCode']== 'AZ']
#df9_NM = df9[df9['StateCode']== 'NM']
#df9_TX = df9[df9['StateCode']== 'TX']
#df9_CA.head(1)
#df9_CA_new = df9_CA[['Data']].sum(axis=1).where(df['MSN'].astype(str).str[0:3], 0)

In [ ]:
import plotly.plotly as py
import ipywidgets as widgets

from ipywidgets import interact, interactive, fixed
from IPython.core.display import HTML
from IPython.display import display, clear_output
from plotly.widgets import GraphWidget


styles = '''<style>.widget-hslider { width: 100%; }
    .widget-hbox { width: 100% !important; }
    .widget-slider { width: 100% !important; }</style>'''

HTML(styles)

#this widget will display our plotly chart
graph = GraphWidget("https://plot.ly/~chriddyp/674")
fig = py.get_figure("https://plot.ly/~chriddyp/674")

#find the range of the slider.
xmin, xmax = fig['layout']['xaxis']['range']

# use the interact decorator to tie a widget to the listener function
@interact(y=widgets.FloatRangeSlider(min=xmin, max=xmax, step=(xmax-xmin)/1000.0, continuous_update=False))
def update_plot(y):
    graph.relayout({'xaxis.range[0]': y[0], 'xaxis.range[1]': y[1]})
    
#display the app    
graph